## Advent of Code 2024

### Day 20: Race Condition

#### Importing libraries

In [38]:
from collections import deque
from collections import defaultdict

#### Loading example and input file

In [35]:
with open('example1.txt') as input_file:
    lines_ex1 = input_file.readlines()

# with open('example2.txt') as inputFile:
#     lines_ex2 = inputFile.readlines()

with open('input.txt') as input_file:
    lines = input_file.readlines()

#### Common functions

In [36]:
def parse(lines):
	start = None
	end = None
	track = set()
	walls = set()
	for i in range(len(lines)):
		for j in range(len(lines[i])):
			if lines[i][j] == '#':
				walls.add((i, j))
			elif lines[i][j] == '.':
				track.add((i, j))
			elif lines[i][j] == 'S':
				start = (i, j)
				track.add((i, j))
			elif lines[i][j] == 'E':
				end = (i, j)
				track.add((i, j))
	return start, end, track, walls

def find_shortest_path(maze, start, end):
	queue = deque([(start, 0)])
	visited = set()

	while queue:
		position, steps = queue.popleft()
		if position == end:
			return steps
		visited.add(position)

		for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
			neighbor = (position[0] + dx, position[1] + dy)
			if neighbor in maze and neighbor not in visited:
				visited.add(neighbor)
				queue.append((neighbor, steps + 1))

	return None

#### Part One

In [33]:
def part_one(input_lines, saved_time=100):
	start, end, track, walls = parse(input_lines)

	potential_cheats = set()
	for i in range(len(input_lines)):
		for j in range(len(input_lines[i])):
			if (i, j) not in track and (((i - 1, j) in track and (i + 1, j) in track) or ((i, j - 1) in track and (i, j + 1) in track)):
				potential_cheats.add((i, j))
	
	track_length = len(track) - 1
	good_cheats = set()
	
	for cheat in potential_cheats:
		track.add(cheat)
		if find_shortest_path(track, start, end) <= track_length - saved_time:
			good_cheats.add(cheat)
		track.remove(cheat)

	return len(good_cheats)

print("Example input: " + str(part_one(lines_ex1, 1)))
print("Real input: " + str(part_one(lines)))

Example input: 44
Real input: 1511


#### Part Two

In [95]:
def manhattan_dist(a, b):
	return abs(a[0] - b[0]) + abs(a[1] - b[1])

def get_ordered_track(start, end, track):
	ordered_track = [start]
	while ordered_track[-1] != end:
		for dx, dy in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
			next_step = (ordered_track[-1][0] + dx, ordered_track[-1][1] + dy)
			if next_step not in ordered_track and next_step in track:
				ordered_track.append(next_step)
	return ordered_track

def part_two(input_lines, saved_time=100):
	start, end, track, walls = parse(input_lines)
	
	ordered_track = get_ordered_track(start, end, track)

	cheats = defaultdict(int)
	for i in range(len(ordered_track) - 1):
		for j in range(i + 1, len(ordered_track)):
			man_dist = manhattan_dist(ordered_track[i], ordered_track[j])
			track_dist = j - i
			if man_dist <= 20 and man_dist < track_dist:
				saved_by_cheat = track_dist - man_dist
				if saved_time <= saved_by_cheat:
					cheats[saved_by_cheat] += 1

	return sum(cheats.values())	# should be 285 for example

print("Example input: " + str(part_two(lines_ex1, 50)))
print("Real input: " + str(part_two(lines)))

Example input: 285
Real input: 1020507
